In [59]:
import os
import sys
sys.path.append(os.path.join('..', 'src'))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from genes_annotations import *
from autoencoder import *
from utils import *

In [2]:
filename = "../data_tisch2/Glioma_GSE102130_expression.h5"

h5 = read_h5(filename)

selected_genes = get_all_genes(h5)

In [3]:
genes_go, all_go, all_genes = get_all_gene_annotations(h5)

HMS:0:00:08.476980 606,840 annotations READ: ../goa_human.gaf 
../go.obo: fmt(1.2) rel(2021-02-01) 50,515 GO Terms; optional_attrs(consider replaced_by)


In [4]:
mask = build_mask(h5,genes_go,all_go,all_genes)

In [5]:
expr_mat = build_expr_mat(h5)
expr_mat = select_gene_expr_v2(h5, expr_mat, all_genes)

'build_expr_mat'  421.57 ms
'select_gene_expr_v2'  335.68 ms


### Récupérer les labels

In [6]:
df = pd.read_csv('../data_tisch2/Glioma_GSE102130_CellMetainfo_table.tsv', delimiter='\t')
targets = np.array(df['Cluster'])
N_classes = targets.max()+1

In [7]:
np.sum(targets==1)

488

### Entrainement sur le groupe 0

In [8]:
dataset0 = expr_mat[targets==0, :]

In [9]:
ae0, train_loader0, embeddings0 = ae_pipeline(mask, dataset0, all_genes, all_go, n_epochs=5, batch_size=5, print_loss=100, output_file='model_0_bis.pth', embed_file='embeddings_ae_0_bis.csv')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
              Tanh-1              [-1, 1, 1414]               0
      MaskedLinear-2              [-1, 1, 1414]               0
            Linear-3               [-1, 1, 500]         707,500
              Tanh-4               [-1, 1, 500]               0
            Linear-5              [-1, 1, 1414]         708,414
              Tanh-6              [-1, 1, 1414]               0
              Tanh-7             [-1, 1, 10232]               0
      MaskedLinear-8             [-1, 1, 10232]               0
Total params: 1,415,914
Trainable params: 1,415,914
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.04
Forward/backward pass size (MB): 0.21
Params size (MB): 5.40
Estimated Total Size (MB): 5.65
----------------------------------------------------------------
[1,     1] loss: 0.92065
[1,   101] lo

### Entrainement sur le groupe 1

In [46]:
dataset1= expr_mat[targets==1, :]

In [49]:
ae, train_loader, embeddings = ae_pipeline(mask, dataset1, all_genes, all_go, n_epochs=5, batch_size=5, print_loss=100, output_file='model_1.pth', embed_file='embeddings_ae_1.csv')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
              Tanh-1              [-1, 1, 1414]               0
      MaskedLinear-2              [-1, 1, 1414]               0
            Linear-3               [-1, 1, 500]         707,500
              Tanh-4               [-1, 1, 500]               0
            Linear-5              [-1, 1, 1414]         708,414
              Tanh-6              [-1, 1, 1414]               0
              Tanh-7             [-1, 1, 10232]               0
      MaskedLinear-8             [-1, 1, 10232]               0
Total params: 1,415,914
Trainable params: 1,415,914
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.04
Forward/backward pass size (MB): 0.21
Params size (MB): 5.40
Estimated Total Size (MB): 5.65
----------------------------------------------------------------
[1,     1] loss: 0.97542
[2,     1] lo

In [17]:
ae1, train_loader1, embeddings1 = ae, train_loader, embeddings

In [10]:
ae0

GeneAutoEncoder(
  (encoder): Sequential(
    (0): MaskedLinear(
      (activation): Tanh()
    )
    (1): Linear(in_features=1414, out_features=500, bias=True)
    (2): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=500, out_features=1414, bias=True)
    (1): Tanh()
    (2): MaskedLinear(
      (activation): Tanh()
    )
  )
)

In [34]:
ae0.get_sorted_genes(45)

[('TBXT', 1.3344258069992065),
 ('SMO', 1.3265795707702637),
 ('TBX19', 1.1676260232925415),
 ('EOMES', 0.9587002992630005),
 ('TBX4', 0.8970722556114197),
 ('TBR1', 0.8322638869285583),
 ('TBX2', 0.7771263122558594),
 ('TBX1', 0.7703359127044678),
 ('TBX3', 0.6920588612556458),
 ('TBX10', 0.6038128733634949),
 ('NOTCH1', 0.3189584016799927),
 ('TBX15', 0.09548897296190262),
 ('APC2', -0.10250582545995712),
 ('APC', -0.33649662137031555),
 ('PRDM14', -0.7197322845458984),
 ('TBX21', -0.7427435517311096),
 ('CDON', -0.8807424306869507),
 ('TBX18', -0.8862106204032898),
 ('DHH', -0.8881945013999939),
 ('MGA', -1.0910060405731201),
 ('PSEN1', -1.1790727376937866),
 ('FKBP8', -1.184144139289856),
 ('TBX6', -1.246217966079712),
 ('SOX9', -1.406280755996704),
 ('TBX5', -1.6267459392547607)]

In [28]:
ae0.encoder[0].W.detach()

tensor([[ 0.0200, -0.0731,  0.1153,  ..., -1.0692,  0.6816,  0.9931],
        [-0.4616, -1.7603,  1.0751,  ...,  0.0207,  1.5827,  1.3610],
        [-1.0472,  0.7226,  1.6309,  ...,  0.7128,  0.2539, -0.1556],
        ...,
        [ 0.1760,  1.3726, -1.5686,  ...,  2.5391,  0.2645,  1.0555],
        [-0.6496,  1.4782, -0.7362,  ..., -0.5032, -0.4651,  0.5863],
        [ 0.1403, -0.8721,  0.3371,  ..., -0.6045,  0.4757, -0.1975]])

In [33]:
all_go[2]

'GO:0000149'

In [38]:
i=all_genes.index('TFEB')

In [42]:
ae0.get_sorted_terms(50)

[('GO:0008474', -0.5287498235702515)]

In [53]:
dict(ae.get_sorted_terms(i))

{'GO:0019899': 1.5770986080169678,
 'GO:0000981': 0.7112523317337036,
 'GO:0006959': 0.3154316246509552,
 'GO:0002250': 0.11283542215824127,
 'GO:0001892': -0.14081084728240967,
 'GO:0006914': -0.5266640186309814,
 'GO:0046983': -1.8595452308654785}

In [52]:
dict(ae0.get_sorted_terms(i))

{'GO:0002250': 0.8904405236244202,
 'GO:0046983': 0.5321925282478333,
 'GO:0019899': 0.23537079989910126,
 'GO:0001892': 0.057046208530664444,
 'GO:0006959': -0.7017244696617126,
 'GO:0000981': -0.7411890029907227,
 'GO:0006914': -1.3758678436279297}

In [54]:
godag = GODag("../go.obo", optional_attrs={'consider', 'replaced_by'}, load_obsolete=True)

../go.obo: fmt(1.2) rel(2021-02-01) 50,515 GO Terms; optional_attrs(consider replaced_by)


In [57]:
godag['GO:0002250'].name

'adaptive immune response'

In [60]:
goterms=GOTerm()

HMS:0:00:10.702523 606,840 annotations READ: ../goa_human.gaf 
../go.obo: fmt(1.2) rel(2021-02-01) 50,515 GO Terms; optional_attrs(consider replaced_by)


In [61]:
goterms.get_go_name('GO:0002250')

'adaptive immune response'

In [62]:
for t, v in ae.get_sorted_terms(i):
    print(goterms.get_go_name(t), v)

enzyme binding 1.5770986080169678
DNA-binding transcription factor activity, RNA polymerase II-specific 0.7112523317337036
humoral immune response 0.3154316246509552
adaptive immune response 0.11283542215824127
embryonic placenta development -0.14081084728240967
autophagy -0.5266640186309814
protein dimerization activity -1.8595452308654785
